In [ ]:
!pip install transformers torch pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [2]:
train_file_path = '/content/generated_flight_data.json'
test_file_path = '/content/test.json'

# Đọc các file JSON vào DataFrame tương ứng
train_df = pd.read_json(train_file_path)
test_df = pd.read_json(test_file_path)

# Đổi tên cột cho cả hai DataFrame
train_df.rename(columns={'Text': 'text', 'Label': 'label'}, inplace=True)
test_df.rename(columns={'Text': 'text', 'Label': 'label'}, inplace=True)

print("Dữ liệu train đã tải:")
print(train_df.head())
print("\nDữ liệu test đã tải:")
print(test_df.head())



Dữ liệu train đã tải:
                                                text    label
0  tôi muốn bay một chuyến khứ hồi từ đà nẵng đến...   flight
1  giá vé khứ hồi từ đà nẵng đến vinh dưới 2 triệ...  airfare
2  cho tôi biết các chuyến bay đến đà nẵng vào ng...   flight
3  những chuyến bay nào khởi hành từ thành phố hồ...   flight
4  hãng hàng không nào có đường bay từ thượng hải...  airline

Dữ liệu test đã tải:
                                                text    label
0    tôi cần một chuyến bay từ cần thơ đến hải phòng   flight
1  những bữa ăn nào được phục vụ trên chuyến bay ...     meal
2  những bữa ăn nào được phục vụ trên chuyến bay ...     meal
3  giá vé một chiều đắt nhất giữa huế và phú quốc...  airfare
4  những hãng hàng không nào có chuyến bay giữa h...  airline


In [3]:
all_labels = pd.concat([train_df['label'], test_df['label']]).unique()
label2id = {label: i for i, label in enumerate(all_labels)}
id2label = {i: label for i, label in enumerate(all_labels)}

# Thêm cột 'label_id' vào cả hai DataFrame
train_df['label_id'] = train_df['label'].map(label2id)
test_df['label_id'] = test_df['label'].map(label2id)

# Chuyển đổi về list để xử lý (KHÔNG CẦN CHIA DỮ LIỆU)
train_texts = train_df['text'].tolist()
train_labels = train_df['label_id'].tolist()
test_texts = test_df['text'].tolist()
test_labels = test_df['label_id'].tolist()

print(f"\nSố lượng mẫu train: {len(train_texts)}")
print(f"Số lượng mẫu test: {len(test_texts)}")
print(f"Các nhãn: {label2id}")


Số lượng mẫu train: 9399
Số lượng mẫu test: 2022
Các nhãn: {'flight': 0, 'airfare': 1, 'airline': 2, 'ground_service': 3, 'quantity': 4, 'city': 5, 'airfare#flight': 6, 'abbreviation': 7, 'aircraft': 8, 'distance': 9, 'ground_fare': 10, 'capacity': 11, 'flight_time': 12, 'meal': 13, 'flight#flight_time': 14, 'aircraft#flight#flight_no': 15, 'city#flight_time': 16, 'restriction': 17, 'flight_no': 18, 'airport': 19, 'flight_no#flight_time': 20, 'flight#flight_no': 21, 'airline#flight_no': 22, 'ground_fare#ground_service': 23, 'aircraft#flight': 24, 'airline#flight': 25}


In [4]:
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"\nModel đang chạy trên: {device}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]


Model đang chạy trên: cuda


In [5]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
train_dataset = IntentDataset(train_encodings, train_labels)
test_dataset = IntentDataset(test_encodings, test_labels)

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [9]:
print("\nBắt đầu quá trình fine-tuning...")
trainer.train()
print("Hoàn tất huấn luyện!")


Bắt đầu quá trình fine-tuning...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.469000,0.623304,0.844708,0.840192,0.840794,0.844708
2,0.167900,0.611735,0.838279,0.834714,0.841588,0.838279
3,0.115500,0.568125,0.861523,0.857812,0.860708,0.861523
4,0.068300,0.566816,0.871909,0.871697,0.874919,0.871909
5,0.047700,0.588194,0.872404,0.871844,0.876001,0.872404


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprec

Hoàn tất huấn luyện!


In [10]:
print("\nĐánh giá model trên tập test:")
eval_results = trainer.evaluate()
print(eval_results)


Đánh giá model trên tập test:


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.5668162703514099, 'eval_accuracy': 0.8719090009891197, 'eval_f1': 0.871696771482093, 'eval_precision': 0.8749187226997782, 'eval_recall': 0.8719090009891197, 'eval_runtime': 5.6398, 'eval_samples_per_second': 358.526, 'eval_steps_per_second': 5.674, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()
    return model.config.id2label[predicted_class_id]
new_sentence = "tr"
predicted_label = predict_intent(new_sentence)
print(f"Câu: '{new_sentence}'")
print(f"==> Ý định dự đoán: '{predicted_label}'")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Câu: 'tr'
==> Ý định dự đoán: 'flight'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_directory = "/content/drive/MyDrive/my_phobert_intent_model"
# 1. Lưu model
model.save_pretrained(save_directory)
# 2. Lưu tokenizer
tokenizer.save_pretrained(save_directory)
print(f"Đã lưu thành công model và tokenizer vào thư mục: {save_directory}")

Đã lưu thành công model và tokenizer vào thư mục: /content/drive/MyDrive/my_phobert_intent_model
